# Task Two: Flow, Density, and Speed Contours

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Define file path

In [90]:
# Group 2 US101SB
stations_us101_g2_path = "Group 2 US101SB/Stations_US101.csv"
zone_readings_10_19_path = "Group 2 US101SB/Zone_Readings_10-19.csv"
zone_readings_10_26_path = "Group 2 US101SB/Zone_Readings_10-26.csv"

# SR_91_Bottleneck_B_150-200
inventory_sr91_b_path = "SR_91_Bottleneck_B_150-200/Inventory.csv"
zone_readings_0831_0901_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0831-0901.csv"
zone_readings_0907_0908_path = "SR_91_Bottleneck_B_150-200/Zone_Readings_0907-0908.csv"

## 2. Load data

In [91]:
# Read all zoning readings data

# USB 101 data
zone_readings_10_19 = pd.read_csv(zone_readings_10_19_path)
zone_readings_10_26 = pd.read_csv(zone_readings_10_26_path)

# SR 91 data
zone_readings_0831_0901 = pd.read_csv(zone_readings_0831_0901_path)
zone_readings_0907_0908 = pd.read_csv(zone_readings_0907_0908_path)

## 3. Explore data

In [92]:
# Drop the index
zone_readings_10_19 = zone_readings_10_19.drop(columns=['index'])
zone_readings_10_19.head()

# Rename the sr 91 data
zone_readings_0831_0901 = zone_readings_0831_0901.rename(columns={'measurement_start': 'date', 'zone_id': 'station_ID'})
zone_readings_0907_0908 = zone_readings_0907_0908.rename(columns={'measurement_start': 'date', 'zone_id': 'station_ID'})

In [93]:
# Calculate the volume in veh/hr/ln
zone_readings_10_19['volume'] = zone_readings_10_19['volume'] * 3600 * (1/30) / zone_readings_10_19['lanes']  # 30 seconds interval, 5 lanes
zone_readings_10_26['volume'] = zone_readings_10_26['volume'] * 3600 * (1/30) / zone_readings_10_26['lanes'] # 30 seconds interval, 5 lanes
zone_readings_0831_0901['volume'] = zone_readings_0831_0901['volume'] * 3600 * (1/60) * (1/2) # 60 seconds interval, 2 lanes
zone_readings_0907_0908['volume'] = zone_readings_0907_0908['volume'] * 3600 * (1/60) * (1/2) # 60 seconds interval, 2 lanes

# Drop the lanes column
zone_readings_10_19 = zone_readings_10_19.drop(columns=['lanes'])
zone_readings_10_26 = zone_readings_10_26.drop(columns=['lanes'])

In [94]:
# Calculate the density = (52.8 / (l_v * l_D)) * occupancy
lv = 18 # ft
l_D = 6 # ft
zone_readings_10_19['density'] = (5280 / (lv + l_D)) * zone_readings_10_19['occupancy']
zone_readings_10_26['density'] = (5280 / (lv + l_D)) * zone_readings_10_26['occupancy']
zone_readings_0831_0901['density'] = (5280/ (lv + l_D)) * zone_readings_0831_0901['occupancy'] * 0.01
zone_readings_0907_0908['density'] = (5280 / (lv + l_D)) * zone_readings_0907_0908['occupancy'] * 0.01
zone_readings_10_19.head()

,date,station_ID,volume,speed,occupancy,density
0,2019-10-19 00:00:00,402005,0.0,80.000,0.000000,0.000000
1,2019-10-19 00:00:00,405389,0.0,80.000,0.000000,0.000000
2,2019-10-19 00:00:00,405390,960.0,75.375,0.012800,2.816000
3,2019-10-19 00:00:00,405394,0.0,80.000,0.000000,0.000000
4,2019-10-19 00:00:00,405395,120.0,69.000,0.004467,0.982667


In [95]:
# Select station_ID = '405390'
station = zone_readings_10_19.loc[zone_readings_10_19['station_ID'] == 405390]
station.head()

,date,station_ID,volume,speed,occupancy,density
2,2019-10-19 00:00:00,405390,960.0,75.375000,0.012800,2.816000
147,2019-10-19 00:00:00,405390,120.0,61.500000,0.004467,0.982667
292,2019-10-19 00:01:00,405390,360.0,62.666667,0.005025,1.105500
437,2019-10-19 00:01:00,405390,960.0,68.125000,0.013075,2.876500
582,2019-10-19 00:02:00,405390,480.0,58.500000,0.012650,2.783000


## 4. Define function to create contour

In [96]:
# Genrate countour of given metrics, the metrics are volume, speed, density.
# Sort data by station_ID, date.
# Get metric value of each station_ID.
# Create a new contour_df, row is date, column is station_ID, value is metric value.
def create_contour(df, metric):
    df = df.sort_values(by=['station_ID', 'date']).reset_index(drop=True)
    station_IDs = df['station_ID'].unique()
    for station_ID in station_IDs:
        station = df.loc[df['station_ID'] == station_ID]
        station = station.reset_index(drop=True)
        if station_ID == station_IDs[0]:
            contour_df = pd.DataFrame({'date': station['date'], station_ID: station[metric]})
        else:
            station_metric = pd.DataFrame({station_ID: station[metric]})
            contour_df = pd.concat([contour_df, station_metric], axis=1)

    return contour_df

# Test for function
metrics = ['volume', 'speed', 'density']
contour = create_contour(zone_readings_10_19, metrics[2])
contour.tail()

,date,400000,400011,400143,400304,400382,400392,400460,400513,400525,...,410553,412439,412443,412446,415697,415704,416620,416911,418244,421965
2868,2019-10-19 23:58:00,5.258,4.1536,2.0130,3.219333,6.724667,2.2605,6.6000,4.282667,3.1460,...,1.716000,4.466,3.8170,1.408,0.0,0.0,7.458,0.000,4.282667,4.605333
2869,2019-10-19 23:58:00,3.366,3.3000,4.5815,5.230500,3.608000,3.0140,4.6145,5.016000,4.6728,...,3.245000,2.574,2.5355,3.058,0.0,0.0,3.058,0.000,1.474000,0.777333
2870,2019-10-19 23:59:00,NaN,5.2756,3.6432,3.747333,9.108000,5.2250,2.5300,4.444000,2.8424,...,3.791333,1.045,2.0020,1.408,0.0,0.0,8.074,1.496,8.235333,3.344000
2871,2019-10-19 23:59:00,0.000,3.6432,1.4080,3.360500,3.300000,2.8105,5.4945,4.774000,3.4716,...,1.510667,3.553,2.2330,2.134,0.0,0.0,3.674,0.000,4.363333,3.872000
2872,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 5. Process the data

In [97]:
metrics = ['volume', 'speed', 'density']

# Create contour file for US101SB 10-19
for metric in metrics:
    contour_df = create_contour(zone_readings_10_19, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-19.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for US101SB 10-26
for metric in metrics:
    contour_df = create_contour(zone_readings_10_26, metric)
    contour_csv_path = f"Group 2 US101SB/{metric}_contour_10-26.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for SR91 0831-0901
for metric in metrics:
    contour_df = create_contour(zone_readings_0831_0901, metric)
    contour_csv_path = f"SR_91_Bottleneck_B_150-200/{metric}_contour_0831-0901.csv"
    contour_df.to_csv(contour_csv_path, index=False)

# Create contour file for SR91 0907-0908
for metric in metrics:
    contour_df = create_contour(zone_readings_0907_0908, metric)
    contour_csv_path = f"SR_91_Bottleneck_B_150-200/{metric}_contour_0907-0908.csv"
    contour_df.to_csv(contour_csv_path, index=False)
